<a href="https://colab.research.google.com/github/ctix/chinese-stock-api/blob/master/simplify_class_and_test_fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The new way to make the structure 
---------
Study for modification  \\
Copy from cstock directory base_engine.py 

In [0]:
import abc

class Engine(object):
    """Base Engine class, must be inherited
    """

    __slots__ = ['_url']

    def __init__(self, base_url=None):
        if base_url is None:
            self._url = self.DEFAULT_BASE_URL
        else:
            self._url = base_url
 
    @abc.abstractmethod
    def parse(self, data, stock_id):
        """
        parse the data from service URL
        :param data:             payload string
        :type data:            ``str``
        :param stock_id:         raw stock/fund id
        :type stock_id:        ``str``
        :returns:            tuple of Stock objects
        :rtype:            ``tuple``
        """
        pass

    def get_url(self, stock_id, date=None):
        """
        transform stock_id date into service URL
        :param stock_id:            stock id  
        :type stock_id:            ``str``
        :param date:            tuple of start date & end date, optional
            e.g. ('2014-12-13', '2014-12-15')
        :type date:            ``tuple``
        :returns:            service URL
        :rtype:            ``str``
        """
        engine_id = self.get_engine_id(stock_id)
        return self._url % engine_id

    def get_engine_id(self, stock_id):
        """
        transform raw stock_id into service stock id
        we regard stock/fund starting with 0 or 3 belongs to shenzhen
        :param stock_id:            raw stock id
        :type stock_id:            ``str``
        :returns:            service stock id
        :rtype:            ``str``
        """
        if stock_id.startswith('0') or stock_id.startswith('3'):
            return self.shenzhen_transform(stock_id)
        
        if stock_id.startswith('6'):
            return self.shanghai_transform(stock_id)
        
        raise ParserException("Unknow stock id %s" % stock_id)

## Copy from the cstock/model.py

In [0]:
class ParserException(Exception):
    pass


class Stock(object):
    
    # yesterday_close is yesterday close price
    # close is today close price

    # volume: unit of stock transacted
    # turnover: total transaction money

    __slots__ = [
        'name',
        'code',
        'date',
        'time',
        'price',
        'open',
        'close',
        'high',
        'low',
        'volume',
        'turnover',
        'yesterday_close',
    ]

    def __init__(self, **argvs):
        
        for (k, v) in argvs.items():
            setattr(self, k, v)

    def as_dict(self):
        result = {
            i: getattr(self, i, None)
            for i in self.__slots__
        }

        # dispose date and time because they are datetime class instance
        if result['date'] is not None:
            result['date'] = str(result['date'])

        if result['time'] is not None:
            result['time'] = str(result['time'])

        return result

13.80738004562425
18.19993988938782
0.9728675345252052


In [0]:
# ##chinese-stock-api/cstock/request.py

import urllib2


class Requester(object):
    """ Requester class for diferent engine
    """

    __slots__ = ["_engine"]

    def __init__(self, engine):

        self._engine = engine

    def request(self, stock_id, date=None):
        """request by stock id and date
        :param stock_id:
            stock id string
        :type stock_id:
            ``str``
        :param date:
            tuple of start date and stop date
            e.g. ('2014-03-04', '2014-03-05')
        :type date:
            ``tuple``
        :returns:
            tuple of stock objects
        :rtype:
            ``tuple``
        """
        
        stock_url = self._engine.get_url(stock_id, date)
        print stock_url

        request = urllib2.Request(stock_url)
        request.add_header('Content-Type', 'application/json')
        response = urllib2.urlopen(request)
        data = response.read()

        return self._engine.parse(data, stock_id)

        0
0   Geeks
1     For
2   Geeks
3      is
4  portal
5     for
6   Geeks


    Name  Age
0    Tom   20
1   nick   21
2  krish   19
3   jack   18


###  api/sina_engine.py

In [0]:
import re
import json
import datetime

from cstock.base_engine import Engine
from cstock.model import Stock, ParserException

class SinaEngine(Engine):
    """
    Sina Engine transform stock id & parse data
    """

    DEFAULT_BASE_URL = "http://hq.sinajs.cn/list=%s"

    def __init__(self, base_url=None):
        
        super(SinaEngine, self).__init__(base_url)

        self.shanghai_transform = lambda sid: "sh%s" % sid
        self.shenzhen_transform = lambda sid: "sz%s" % sid

    def get_url(self, stock_id, date=None):
        if date is not None:
            raise ParserException("Sina Engie does not accept date")

        return super(SinaEngine, self).get_url(stock_id)

    def parse(self, data, stock_id):

        def prepare_data(data):
            """because sina does not return a standard data,
            we need to extract the real data part
            """
            regroup = re.match(r'^var.*="(.*)"', data)

            if regroup:
                return regroup.group(1)
            else:
                raise ParserException("Unable to extact json from %s" % data)

        data_string = prepare_data(data)
        obj = data_string.split(',')
        return (self._generate_stock(obj, stock_id),)

    @staticmethod
    def _generate_stock(obj, stock_id):
        d = dict(enumerate(obj))

        date = d.get(30, None)
        time = d.get(31, None)

        if date is not None:
            date = datetime.datetime.strptime(date, '%Y-%m-%d').date()

        if time is not None:
            time = datetime.datetime.strptime(time, '%H:%M:%S').time()

        return Stock(
            code=stock_id,
            name=d.get(0, None),
            open=d.get(1, None),
            yesterday_close=d.get(2, None),
            price=d.get(3, None),
            high=d.get(4, None),
            low=d.get(5, None),
            volume=d.get(8, None),
            turnover=d.get(9, None),
            date=date,
            time=time,
        )

     Name Qualification
0     Jai           Msc
1  Princi            MA
2  Gaurav           MCA
3    Anuj           Phd


### Test the sina engine 

In [0]:
# system library
import unittest
import datetime

# project library
from cstock.sina_engine import SinaEngine


class TestEngine(unittest.TestCase):

    def setUp(self):
        self.engine = SinaEngine()

    def test_get_url(self):
        url = self.engine.get_url('600010') 
        self.assertEqual(url, "http://hq.sinajs.cn/list=sh600010")

    def test_parse(self):
        data = 'var hq_str_sh600010="包钢股份,5.98,5.99,6.15,6.34,5.95,6.15,6.16,542628944,3341837325,239800,6.15,1033800,6.14,2408711,6.13,1719525,6.12,1001900,6.11,2873590,6.16,1481300,6.17,2113716,6.18,1177600,6.19,2587603,6.20,2015-03-18,15:03:05,00";'

        stock = self.engine.parse(data, 'foo_id')
        self.assertEqual(len(stock), 1)
        self.assertEqual(
            stock[0].as_dict(),
            {'yesterday_close': '5.99',
             'close': None,
             'code': 'foo_id',
             'high': '6.34',
             'low': '5.95',
             'name': '\xe5\x8c\x85\xe9\x92\xa2\xe8\x82\xa1\xe4\xbb\xbd',
             'open': '5.98',
             'price': '6.15',
             'turnover': '3341837325',
             'volume': '542628944',
             'date': '2015-03-18',
             'time': '15:03:05'}
        )

###  Test the  dataSource requst

In [0]:
# system library
import unittest
from datetime import datetime

# project library
from cstock.hexun_engine import HexunEngine
from cstock.sina_engine import SinaEngine
from cstock.yahoo_engine import YahooEngine
from cstock.request import Requester
from cstock.model import Stock

class TestRequester(unittest.TestCase):

    def setUp(self):
        engine = HexunEngine()
        self.hexun_requester = Requester(engine)
        engine = SinaEngine()
        self.sina_requester = Requester(engine)
        engine = YahooEngine()
        self.yahoo_requester = Requester(engine)

    def test_hexun_request(self):
        stock = self.hexun_requester.request('000626')
        self.assertEqual(len(stock), 1)
        self.assertEqual(stock[0].__class__, Stock)

    def test_sina_request(self):
        stock = self.sina_requester.request('002475')
        self.assertEqual(len(stock), 1)
        self.assertEqual(stock[0].__class__, Stock)

    def test_yahoo_request(self):
        stock = self.yahoo_requester.request('002475',
                                             ('2015-03-04', '2015-03-05'))
        self.assertEqual(len(stock), 2)
        self.assertEqual(stock[0].__class__, Stock)

,First Score,Second Score,Third Score
0,100.0,30.0,0.0
1,90.0,45.0,40.0
2,0.0,56.0,80.0
3,95.0,0.0,98.0


     name  degree  score
0  aparna     MBA     90
1  pankaj     BCA     40
2  sudhir  M.Tech     80
3   Geeku     MBA     98


### Test the 

In [0]:
# chinese-stock-api/test/test_model.py
# system library
import unittest
import datetime

# project library
from cstock.model import Stock


class TestModel(unittest.TestCase):
    
    def test_stock(self):
        stock = Stock(
            code='000626',
            time=datetime.time(3, 15, 0),
            price=21.5,
            open=31.5,
            close=30,
            low=12
        )
        self.assertEqual(
            stock.as_dict(),
            {'high': None, 'low': 12, 'open': 31.5, 'code': '000626',
             'price': 21.5, 'time': '03:15:00', 'close': 30, 'date': None,
             'volume': None, 'turnover': None, 'name': None,
             'yesterday_close': None}
        )

sudhir
M.Tech
80
